In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Below is the Dataset (top 5 entries) which we will be using for training the Logistic Regression model.

It takes 6 variables namely :

**-->Year**

**-->GDP per capita (current US$)**

**-->GDP growth (annual %)**

**-->Inflation,consumer prices (annual %)**

**-->Population growth (annual %)**

**-->Life expectancy at birth, total (years)**

**-->Unemployement rate**





In [58]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Unemployement_pred_dataset.xlsx',)
df.head()


,Year,GDP per capita (current US$),GDP growth (annual %),"Inflation, consumer prices (annual %)",Population growth (annual %),"Life expectancy at birth, total (years)",Unemployement rate
0,1991,303.0,1.06,13.87,2.12,58.15,6.737
1,1992,317.0,5.48,11.79,2.10,58.63,6.815
2,1993,301.0,4.75,6.33,2.07,59.12,6.798
3,1994,346.0,6.66,10.25,2.04,59.59,6.830
4,1995,374.0,7.57,10.22,2.01,60.06,7.014


*Importing Necessary Libraries and Defining the dataframes*

Now Importing the Logistic regression Model from 'sklearn' python libraray and importing other necessary libraries as well .

Defining dataframe and splitting the dataset into Train and Test data and converting the unemployment rate data into binary form using a threshold value of unemployment rate.

In [68]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

# Define the path to your dataset
dataset_path = '/content/drive/MyDrive/Unemployement_pred_dataset.xlsx'

# Load the dataset
df = pd.read_excel(dataset_path)

# Clean column names (remove leading and trailing whitespace)
df.columns = df.columns.str.strip()

# Drop the row corresponding to year 2020 containing the outliers due to the Covid19 pandemic
df = df.drop(29)

# Extract independent variables (features) and dependent variable (target)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assume 'df' is your DataFrame containing the data

X = df[['Year', 'GDP per capita (current US$)', 'GDP growth (annual %)', 'Inflation, consumer prices (annual %)', 'Population growth (annual %)', 'Life expectancy at birth, total (years)']]
Y = df['Unemployement rate']

# Impute missing values with the mean
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Apply feature scaling
scaler = StandardScaler()
X_imputed = scaler.fit_transform(X_imputed)

# Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X_imputed, Y, test_size=0.2, random_state=21)

# Define the threshold to classify high and low unemployment for training data
threshold_train = np.average(Y_train)

# Convert Unemployment Rate into Binary form for training data
Y_train_category = Y_train.apply(lambda x: 0 if x >= threshold_train else 1)

# Print the threshold value for training data
print('Threshold value of unemployment rate for training data is:', threshold_train)

# Define the threshold to classify high and low unemployment for test data
threshold_test = np.average(Y_test)

# Convert Unemployment Rate into Binary form for test data
Y_test_category = Y_test.apply(lambda x: 0 if x >= threshold_test else 1)

# Print the threshold value for test data
print('Threshold value of unemployment rate for test data is:', threshold_test)

# Print the converted Y_test values into binary form
print('Y_Test - Actual Values of Unemployment Rate during Test Data Years')
print(Y_test_category)

# Now you can use Y_train_category and Y_test_category for model training and testing.



Threshold value of unemployment rate for training data is: 7.854708333333334
Threshold value of unemployment rate for test data is: 7.436499999999999
Y_Test - Actual Values of Unemployment Rate during Test Data Years
5     1
23    0
22    0
28    1
1     1
21    0
Name: Unemployement rate, dtype: int64


*Applying Logistics Regression and calculation of the probability of low unemployement based on the threshold value set using the average value set by the user.*

*Also Calculating the Accuracy Score and F1 score.*

In [69]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Create and fit logistic regression model
logreg_model = LogisticRegression(random_state=21)
logreg_model.fit(X_train, Y_train_category)

# Predictions on training and test sets
train_predictions = logreg_model.predict(X_train)
test_predictions = logreg_model.predict(X_test)

# Convert predicted probabilities to binary form using a threshold of 0.5
train_pred_binary = (logreg_model.predict_proba(X_train)[:, 1] > 0.5).astype(int)
test_pred_binary = (logreg_model.predict_proba(X_test)[:, 1] > 0.5).astype(int)

# Print metrics for training set
print("Training Set Metrics:")
print("Accuracy:", accuracy_score(Y_train_category, train_pred_binary))
print("Precision:", precision_score(Y_train_category, train_pred_binary))
print("Recall:", recall_score(Y_train_category, train_pred_binary))
print("F1 Score:", f1_score(Y_train_category, train_pred_binary))
print("ROC AUC Score:", roc_auc_score(Y_train_category, logreg_model.predict_proba(X_train)[:, 1]))

# Print metrics for test set
print("\nTest Set Metrics:")
print("Accuracy:", accuracy_score(Y_test_category, test_pred_binary))
print("Precision:", precision_score(Y_test_category, test_pred_binary))
print("Recall:", recall_score(Y_test_category, test_pred_binary))
print("F1 Score:", f1_score(Y_test_category, test_pred_binary))
print("ROC AUC Score:", roc_auc_score(Y_test_category, logreg_model.predict_proba(X_test)[:, 1]))


Training Set Metrics:
Accuracy: 0.875
Precision: 1.0
Recall: 0.75
F1 Score: 0.8571428571428571
ROC AUC Score: 0.9027777777777778

Test Set Metrics:
Accuracy: 0.8333333333333334
Precision: 1.0
Recall: 0.6666666666666666
F1 Score: 0.8
ROC AUC Score: 1.0
